# Use of Measured Aerosol Optical Depth and Precipitable Water to Model Clear Sky Irradiance
*presented at: 2017 IEEE PVSC-44*
*by: Mark A. Mikofski, Clifford W. Hansen, William F. Holmgren and Gregory M. Kimball*

## Analysis
Predicting irradiance is crucial for developing solar power systems. Clear sky models are used to predict irradiance based on solar position and atmospheric data. This paper compares clear sky predictions using atmospheric data from ECMWF with irradiance measurments from SURFRAD. This notebook presents the analysis for the paper and presentation given by the authors at the 2017 IEEE PVSC-44 in Washingtion DC June 25-30th.

### ECMWF
The European Center for Medium Weather Forecast hosts datasets for the EU Copernicus MACC clearsky model. This data set contains aerosol optical depth and total column water vapor data derived from ground measurments and satelite data for the entire globe from 2003 to 2012.

### SURFRAD
SURFRAD data can be viewed online or downloaded from a NOAA FTP site <ftp://aftp.cmdl.noaa.gov/data/radiation/surfrad/>.

In [1]:
# imports and settings
%matplotlib inline

from pvsc44_clearsky_aod import (
    datetime, FTP, logging, os, StringIO, h5py, plt, np, pd, pytz, sns, tzwhere,
    LOGGER, WHERETZ, JAN1, JUN1, tz_latlon
)

![Creative Commons License](https://i.creativecommons.org/l/by/4.0/88x31.png)
Use of Measured Aerosol Optical Depth and Precipitable Water to Model Clear Sky Irradiance by [Mark A. Mikofski, Clifford W. Hansen, William F. Holmgren and Gregory M. Kimball](https://github.com/mikofski/pvsc44-clearsky-aod) is licensed under a [Creative Commons Attribution 4.0 International License](https://creativecommons.org/licenses/by/4.0/).


In [ ]:
data['sxf'][['zen', 'dw_solar', 'direct_n', 'diffuse', 'temp', 'rh', 'windspd', 'winddir', 'pressure']]['2003-06-15':'2003-06-16']

In [ ]:
data['sxf'][['direct_n', 'diffuse']]['2003-06-15':'2003-06-16'].plot()

In [ ]:
%whos

In [ ]:
df[df['qc_dwsolar']==0]

In [ ]:
df.drop('qc_dwsolar',1)